## CAR RADIOS PROBLEM

Dataset: car radios

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import datetime

df = pd.read_excel('/content/data_carradios.xlsx')
df


,perc_defec,bdate,team,training,datep,prizeq,prized
0,0.00,1981-07-02,8,1,2021-07-01,0,0
1,36.32,1992-06-14,6,0,2021-07-02,0,0
2,48.91,2003-05-28,7,0,2021-07-05,500,0
3,20.36,1992-06-14,10,0,2021-07-06,0,0
4,42.07,2003-05-28,7,0,2021-07-07,500,0
...,...,...,...,...,...,...,...
995,21.50,1981-07-02,1,0,2021-12-13,0,0
996,7.53,1981-07-02,2,0,2021-12-14,500,600
997,27.70,1997-12-05,10,0,2021-12-15,0,0
998,0.00,1986-12-23,5,0,2021-12-16,0,600


# 1st STEP

In order to transform the column of `bdate` into `age`.


Aqui estamos só a criar uma função, ele ainda não sabe qual coluna é que nós vamos querer.

`now()` belongs to datetime

In [ ]:
def get_ages(col):
  result = (datetime.datetime.now()-col).astype('<m8[Y]')
  result = pd.DataFrame(result)
  return result


When we need to use a user-defined function, we nedd `FunctionTranformer`.

In [ ]:
from sklearn.preprocessing import FunctionTransformer

`features_names_out`="one-to-one" is to later get the names of the transformed colums

In [ ]:
ager = Pipeline([
    ('ages', FunctionTransformer(get_ages, feature_names_out='one-to-one')),
    ('scale', StandardScaler())
])

# 2nd STEP

Vamos fazer uma nova coluna com os dias da semana.

In [ ]:
def get_weekdays(col):
  result = col.iloc[:,0].dt.weekday
  result = pd.DataFrame(result)
  return result

2ª tabela do tablet

When we have categorical predictors, we need to transform them into binary variables.

In the linear regression model, one of the binary MUST be removed (multicolinearity).

**ONEHOTENCODER**

Encode categorical features as a one-hot numeric array.

The input to this transformer should be an array-like of integers or strings, denoting the values taken on by categorical (discrete) features. The features are encoded using a one-hot (aka ‘one-of-K’ or ‘dummy’) encoding scheme. This creates a binary column for each category and returns a sparse matrix or dense array (depending on the sparse_output parameter)

By default, the encoder derives the categories based on the unique values in each feature. Alternatively, you can also specify the categories manually.

In [ ]:
from sklearn.preprocessing import OneHotEncoder


In [ ]:
weeker = Pipeline([
    ('weekd', FunctionTransformer(get_weekdays, feature_names_out='one-to-one')),
    ('oneh', OneHotEncoder(drop='first'))
])


`tr` stands for transformed

In [ ]:
preprocessor = ColumnTransformer([
    ('ages_tr', ager, ['bdate']),
    ('weekd_tr', weeker, ['datep']),
    ('team_tr', OneHotEncoder(drop='first'), ['team']),
    ('scaler', StandardScaler(), ['prized', 'prizeq'])],
    remainder='passthrough')

`remainder` stands for the remain (o que sobra) e não reminder (de lembrar)

In [ ]:
X = df.drop('perc_defec', axis=1)
y = df['perc_defec']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)

pipe = Pipeline([
    ('pre', preprocessor),
    ('lm', LinearRegression())])

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_train)
y_pred

array([20.08740234, 20.41943359, 10.64794922,  2.07958984, 30.55419922,
       20.18896484, 38.86279297, 20.26904297, 30.61669922, 20.64013672,
       30.55419922, 38.41552734,  1.99755859, 20.46630859, 20.10888672,
       20.63232422, 39.83154297, 20.32177734,  2.44287109, 20.34716797,
       39.83154297, 57.99560547, 20.10888672,  9.65576172,  9.78857422,
       20.40185547,  2.08740234,  1.78466797,  2.14990234, 29.56787109,
       48.71044922, 17.26318359,  9.65576172, 20.46630859, 39.83154297,
       11.29248047, 38.49560547, 30.18896484,  1.45849609, 39.60498047,
        1.91943359,  2.31787109, 21.30029297, 39.60498047, 30.18896484,
       20.56005859, 39.29052734, 39.15576172,  1.45849609, 39.55029297,
       -0.88525391, 20.20458984, 39.97998047,  1.24169922, 39.60498047,
       10.95654297,  1.10107422, 11.13623047, 10.64013672, 19.64599609,
       29.67529297, -0.76025391, 20.33935547,  2.31787109, 11.65966797,
       19.43310547, 19.13818359,  1.10107422, 20.84716797,  1.32

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
mae = mean_absolute_error(y_train, y_pred)
rmse = mean_squared_error(y_train, y_pred, squared=False)
r2 = r2_score(y_train, y_pred)

print(f'MAE = {mae}')
print(f'RMSE = {rmse}')
print(f'R2 = {r2}')

MAE = 3.3389007324218745
RMSE = 4.333033342195751
R2 = 0.9174359688584746


Since the measures of errors are similar for the test set, we can expect good performance of the model in unseen new cases.

# LASSO REGRESSION

dia 20-03

1. Sometimes, some predictors are week, in the sense that the respective coeficient is close to zero, but not zero.
2. When a predictor is week, we should remove it, because:
  - The smaller is the number of predictors, the easier is the interpretation of the model.
  - The larger the number of predictors, the more data is needed to have the model working with good predictive performance.

Lasso Regression sets to zero the weaker predictors automatically.


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Lasso
import datetime

df = pd.read_excel('/content/data_carradios.xlsx')
df

,perc_defec,bdate,team,training,datep,prizeq,prized
0,0.00,1981-07-02,8,1,2021-07-01,0,0
1,36.32,1992-06-14,6,0,2021-07-02,0,0
2,48.91,2003-05-28,7,0,2021-07-05,500,0
3,20.36,1992-06-14,10,0,2021-07-06,0,0
4,42.07,2003-05-28,7,0,2021-07-07,500,0
...,...,...,...,...,...,...,...
995,21.50,1981-07-02,1,0,2021-12-13,0,0
996,7.53,1981-07-02,2,0,2021-12-14,500,600
997,27.70,1997-12-05,10,0,2021-12-15,0,0
998,0.00,1986-12-23,5,0,2021-12-16,0,600


Comparativamente à aula anterior ele apenas importou o `Lasso` e substituiu `lm` por `lasso`.

In [3]:
def get_ages(col):
  result = (datetime.datetime.now()-col).astype('<m8[Y]')
  result = pd.DataFrame(result)
  return result

ager = Pipeline([
    ('ages', FunctionTransformer(get_ages, feature_names_out='one-to-one')),
    ('scale', StandardScaler())
])

def get_weekdays(col):
  result = col.iloc[:,0].dt.weekday
  result = pd.DataFrame(result)
  return result

weeker = Pipeline([
    ('weekd', FunctionTransformer(get_weekdays, feature_names_out='one-to-one')),
    ('oneh', OneHotEncoder(drop='first'))
])

preprocessor = ColumnTransformer([
    ('ages_tr', ager, ['bdate']),
    ('weekd_tr', weeker, ['datep']),
    ('team_tr', OneHotEncoder(drop='first'), ['team']),
    ('scaler', StandardScaler(), ['prized', 'prizeq'])],
    remainder='passthrough')

X = df.drop('perc_defec', axis=1)
y = df['perc_defec']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)

pipe = Pipeline([
    ('pre', preprocessor),
    ('lasso', Lasso(alpha=0.1))])

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_train)
y_pred

array([ 1.95854979e+01,  2.07321659e+01,  1.13849310e+01,  2.30907362e+00,
        2.97110759e+01,  2.07321659e+01,  3.83644755e+01,  2.07321659e+01,
        2.97730735e+01,  2.07603466e+01,  2.97110759e+01,  3.83926562e+01,
        1.95318828e+00,  2.00033808e+01,  2.06983490e+01,  2.07603466e+01,
        3.87497320e+01,  2.07603466e+01,  2.33725437e+00,  2.06983490e+01,
        3.87497320e+01,  5.64440392e+01,  2.06983490e+01,  1.08333274e+01,
        1.08615081e+01,  2.07603466e+01,  1.92500754e+00,  2.24707599e+00,
        2.27525673e+00,  2.91594722e+01,  4.74053831e+01,  1.77057668e+01,
        1.08333274e+01,  2.00033808e+01,  3.87497320e+01,  1.13849310e+01,
        3.83926562e+01,  2.96828951e+01,  1.78565071e+00,  3.87497320e+01,
        2.27525673e+00,  2.30907362e+00,  2.07941635e+01,  3.87497320e+01,
        2.96828951e+01,  2.07603466e+01,  3.84546538e+01,  3.84264731e+01,
        1.78565071e+00,  3.86877344e+01,  1.14595725e-02,  2.06701683e+01,
        3.87779127e+01,  

**ALPHA**

The parameter alpha controls the intensity of penalization of the weaker predictors. The larger the value of alpha the more coeficients will be set to zero.

In [4]:
pipe.named_steps['pre'].get_feature_names_out()

array(['ages_tr__bdate', 'weekd_tr__datep_1', 'weekd_tr__datep_2',
       'weekd_tr__datep_3', 'weekd_tr__datep_4', 'team_tr__team_2',
       'team_tr__team_3', 'team_tr__team_4', 'team_tr__team_5',
       'team_tr__team_6', 'team_tr__team_7', 'team_tr__team_8',
       'team_tr__team_9', 'team_tr__team_10', 'scaler__prized',
       'scaler__prizeq', 'remainder__training'], dtype=object)

In [5]:
pipe.named_steps['lasso'].coef_

array([  0.03130011,  -0.55160365,  -0.        ,  -0.        ,
        17.69430722,  -0.        ,   9.01272689,  -0.        ,
         0.        ,  -0.        ,   8.55764264,  -2.29761405,
         0.        ,  -0.        ,  -8.59784934,   4.31942568,
       -18.42309227])

In [6]:
results=pd.DataFrame(
    {'names':pipe.named_steps['pre'].get_feature_names_out(),
     'estimates':pipe.named_steps['lasso'].coef_}
)
results

,names,estimates
0,ages_tr__bdate,0.031300
1,weekd_tr__datep_1,-0.551604
2,weekd_tr__datep_2,-0.000000
3,weekd_tr__datep_3,-0.000000
4,weekd_tr__datep_4,17.694307
5,team_tr__team_2,-0.000000
6,team_tr__team_3,9.012727
7,team_tr__team_4,-0.000000
8,team_tr__team_5,0.000000
9,team_tr__team_6,-0.000000


NOTE: don't forget that to create dataframes we use `{}` instead of `()`.

Based on the results the company may take action, like reduce the prize.